In [2]:
import numpy as np
import re

In [28]:
rotations = ["G2","G3"]
linear = ["G1","G0"]

def extract_xy(cmd):
    words = cmd.split(" ")
    x,y = None, None
    for w in words:
        if "X" in w:
            x = float(w[1:])
        if "Y" in w:
            y = float(w[1:])
    return x,y

def extract_ij(cmd):
    words = cmd.split(" ")
    i,j = None, None
    for w in words:
        if "I" in w:
            i = float(w[1:])
        if "J" in w:
            j = float(w[1:])
    return i,j

In [42]:
f = open("gcode/tube_path_v2.pgm","r")
lines = f.read().splitlines()
f.close()

preamble = []
moves = []
xs = [] 
ys = []
ivals = []
jvals = []
for l in lines:
    cmd = l.split(" ")[0]
    if cmd not in rotations+linear:
        preamble.append(l)
        continue
    else:
        x,y = extract_xy(l)
        i,j = extract_ij(l)
        if x == None:
            x = xs[-1]
        if y == None:
            y = ys[-1]
        moves.append(cmd)
        xs.append(x)
        ys.append(y)
        ivals.append(i)
        jvals.append(j)

In [44]:
new_moves = []
for k in range(1,len(moves)):
    mv,x,y,i,j = moves[k],xs[k],ys[k],ivals[k],jvals[k]
    if x == None: 
        x = xs[k-1]
    if y == None:
        y = ys[k-1]
    dx = x - xs[k-1]
    dy = y - ys[k-1]
    if mv in linear:
        line = "{0} X{1:.3f} Y{2:.3f}".format(mv,dx,dy)
    else:
        line = "{0} X{1:.3f} Y{2:.3f} I{3:.3f} J{3:.3f}".format(mv,dx,dy,i,j)
    new_moves.append(line)

In [46]:
new_output = preamble+new_moves
fout = open("gcode/tube_path_relative_v2.pgm","w")
for l in new_output:
    if l == "G90":
        l = "G91"
    fout.write(l+"\n")
fout.close()

In [41]:
f = open("gcode/tube_path_explicit.pgm","r")
lines = f.read().splitlines()
f.close()

mvt_lines = []
commands = []
coords = []
rot_centers = []
for l in lines:
    cmd = l.split(" ")[0]
    if cmd not in moves:
        continue
    commands.append(cmd)
    mvt_lines.append(l)
    x,y = extract_coords_2d(l)
    if x == None and y != None:
        x = coords[-1][0]
    if x != None and y == None:
        y = coords[-1][1]
    coords.append([x,y])
    i,j = extract_rotation_centers(l)
    rot_centers.append([i,j])

NameError: name 'extract_coords_2d' is not defined

In [63]:
output = ["G91","ENABLE X Y","G16 X Y Z","G17"]
for k in range(1,len(mvt_lines)):
    l = mvt_lines[k]
    cmd = commands[k]
    dx = coords[k][0] - coords[k-1][0]
    dy = coords[k][1] - coords[k-1][1]
    i,j = rot_centers[k][0],rot_centers[k][1]
    if cmd == "G1":
        line = "G1 X{0:.4f} Y{1:.4f} F40".format(dx,dy)
    if cmd == "G2" or cmd == "G3":
        line = "{0} X{1:.4f} Y{2:.4f} I{3:.4f} J{4:.4f}".format(cmd,dx,dy,i,j)
    output.append(line)
output.append("DISABLE X Y")

In [64]:
fout = open("gcode/tube_path_relative.pgm","w")
for l in output:
    fout.write(l+"\n")
fout.close()

In [65]:
rot_deg = 90
rot = rot_deg*np.pi/180
mtx = np.array([[np.cos(rot),-np.sin(rot)],[np.sin(rot),np.cos(rot)]])
coords_rot = [np.dot(mtx,vec) for vec in coords]
rot_centers_rot = [np.dot(mtx,vec) for vec in rot_centers]
output_rot = ["G91","ENABLE X Y","G16 X Y Z","G17"]
for k in range(1,len(mvt_lines)):
    l = mvt_lines[k]
    cmd = commands[k]
    dx = coords_rot[k][0] - coords_rot[k-1][0]
    dy = coords_rot[k][1] - coords_rot[k-1][1]
    i,j = rot_centers_rot[k][0],rot_centers_rot[k][1]
    if cmd == "G1":
        line = "G1 X{0:.4f} Y{1:.4f} F40".format(dx,dy)
    if cmd == "G2" or cmd == "G3":
        line = "{0} X{1:.4f} Y{2:.4f} I{3:.4f} J{4:.4f}".format(cmd,dx,dy,i,j)
    output_rot.append(line)
output_rot.append("DISABLE X Y")

In [66]:
fout = open("gcode/tube_path_relative_rotate{0}.pgm".format(rot_deg),"w")
for l in output_rot:
    fout.write(l+"\n")
fout.close()